<a href="https://colab.research.google.com/github/Deeksha-HL/Fcae-Detection-processing/blob/main/Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

dataset_path = "/content/drive/MyDrive/archive"   # change if inside a folder
print(os.listdir(dataset_path))


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/archive'

In [ ]:
!pip install mtcnn opencv-python tqdm


In [ ]:
import os
import cv2
import numpy as np
from mtcnn import MTCNN
from tqdm import tqdm



In [ ]:
processed_path = "/content/processed_dataset"
os.makedirs(processed_path, exist_ok=True)


In [ ]:
detector = MTCNN()


In [ ]:
def preprocess_image(image_path, save_path):

    image = cv2.imread(image_path)

    if image is None:
        return

    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = detector.detect_faces(image_rgb)

    if results:
        x, y, w, h = results[0]['box']

        x, y = max(0, x), max(0, y)

        face = image[y:y+h, x:x+w]

        face = cv2.resize(face, (160, 160))

        # ❌ REMOVE normalization here

        cv2.imwrite(save_path, face)



In [ ]:
!rm -rf /content/processed_dataset


In [ ]:
os.makedirs(processed_path, exist_ok=True)


In [ ]:
for root, dirs, files in os.walk(dataset_path):

    for file in tqdm(files):

        if file.lower().endswith(('.jpg', '.jpeg', '.png')):

            input_path = os.path.join(root, file)

            # Maintain same folder structure
            relative_path = os.path.relpath(root, dataset_path)
            save_dir = os.path.join(processed_path, relative_path)
            os.makedirs(save_dir, exist_ok=True)

            save_path = os.path.join(save_dir, file)

            preprocess_image(input_path, save_path)

print("✅ All images processed successfully!")


In [ ]:
import os
print(os.listdir("/content"))


In [ ]:
print(os.listdir("/content/processed_dataset"))


In [ ]:
import os

for root, dirs, files in os.walk("/content/processed_dataset"):
    for file in files:
        if file.lower().endswith((".jpg", ".png", ".jpeg")):
            print(os.path.join(root, file))




In [ ]:
sample_image = "/content/processed_dataset/Selfie & id data - public sample/10/archive_selfies/2_recent.jpeg"
img = cv2.imread(sample_image)


if img is None:
    print("Image not found!")
else:
    import matplotlib.pyplot as plt
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.axis("off")


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)


In [ ]:
class FaceDetector:
    """MTCNN detector with Haar Cascade fallback."""

    def __init__(self):
        if cfg.USE_MTCNN:
            try:
                self.mtcnn = MTCNN()
                self._mode = 'mtcnn'
                print('  ✅  FaceDetector  →  MTCNN loaded')
            except Exception as e:
                print(f'   MTCNN failed ({e}), falling back to Haar')
                self._mode = 'haar'
        else:
            self._mode = 'haar'

        if self._mode == 'haar':
            xml = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
            self.haar = cv2.CascadeClassifier(xml)
            print('  ✅  FaceDetector  →  Haar Cascade loaded')

    def detect(self, img_rgb):
        """Return (x,y,w,h), landmarks_dict or None, None."""
        if self._mode == 'mtcnn':
            return self._mtcnn_detect(img_rgb)
        return self._haar_detect(img_rgb), None

    def _mtcnn_detect(self, img_rgb):
        results = self.mtcnn.detect_faces(img_rgb)
        if not results:
            return None, None
        best = max(results, key=lambda r: r['confidence'])
        if best['confidence'] < cfg.MIN_CONFIDENCE:
            return None, None
        x, y, w, h = best['box']
        x, y = max(0, x), max(0, y)
        return (x, y, w, h), best['keypoints']

    def _haar_detect(self, img_rgb):
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        faces = self.haar.detectMultiScale(gray, 1.1, 5, minSize=(60, 60))
        if len(faces) == 0:
            return None
        return tuple(max(faces, key=lambda f: f[2] * f[3]))
